# TVKit Comprehensive Sample Notebook

**TVKit** is a Python library for TradingView's financial data APIs with real-time WebSocket streaming and comprehensive data export capabilities.

## Features Covered in This Notebook

- **OHLCV Data Fetching** - Historical and real-time financial data
- **Data Export System** - Multiple formats (Polars DataFrame, JSON, CSV)
- **Financial Analysis** - Technical indicators and data analysis
- **Real-time Streaming** - Live market data updates
- **Multi-symbol Operations** - Working with multiple financial instruments

## Prerequisites

```bash
# Install tvkit with all dependencies
pip install tvkit polars matplotlib seaborn
```

In [2]:
# Import required libraries
import asyncio
import nest_asyncio

# TVKit imports
from tvkit.api.chart.ohlcv import OHLCV
from tvkit.export import DataExporter, ExportFormat
from tvkit.api.utils import convert_timestamp_to_iso

# Enable nested event loops for Jupyter notebooks
nest_asyncio.apply()

# Optional: Data analysis and visualization
try:
    import polars as pl
    import matplotlib.pyplot as plt
    import seaborn as sns

    ANALYSIS_AVAILABLE = True
    print("✅ Analysis libraries loaded successfully")

    # Use the imports to avoid F401 warnings
    _ = pl, plt, sns
except ImportError as e:
    ANALYSIS_AVAILABLE = False
    print(f"⚠️  Analysis libraries not available: {e}")

print("🚀 TVKit sample notebook initialized!")

✅ Analysis libraries loaded successfully
🚀 TVKit sample notebook initialized!


```markdown
## Logging Configuration

The following cell configures logging and warning settings to ensure clean, readable notebook output:

- **Suppresses debug and info logs** from libraries such as `httpx`, `websockets`, and others, so only warnings and errors are shown.
- **Disables most warnings** to avoid cluttering the notebook with non-critical messages.
- This setup is recommended for interactive analysis, as it keeps the output focused on results and important issues.

You can adjust the logging level or re-enable warnings if you need more detailed troubleshooting information.
```

In [3]:
# Configure logging to suppress debug messages
import logging
import warnings

# Set logging levels to reduce verbosity
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("websockets").setLevel(logging.WARNING)

# Optionally suppress warnings
warnings.filterwarnings("ignore")

print("🔇 Debug logging disabled - clean output mode enabled")

🔇 Debug logging disabled - clean output mode enabled


## Basic OHLCV Data Fetching

Let's start by fetching historical OHLCV (Open, High, Low, Close, Volume) data for Apple stock (AAPL) from NASDAQ.

In [ ]:
async def fetch_historical_ohlcv_data():
    """Fetch historical OHLCV data for Apple stock."""
    async with OHLCV() as ohlcv:
        # Fetch last 100 daily bars for Apple
        ohlcv_data = await ohlcv.get_historical_ohlcv(
            exchange_symbol="NASDAQ:AAPL",
            interval="1D",  # Daily intervals
            bars_count=100,
        )

    # Display basic information
    print(f"📊 Fetched {len(ohlcv_data)} OHLCV bars")
    print(
        f"📅 Date range: {convert_timestamp_to_iso(ohlcv_data[0].timestamp)} to {convert_timestamp_to_iso(ohlcv_data[-1].timestamp)}"
    )

    # Show first few bars
    print("\n🔍 First 3 bars:")
    for i, bar in enumerate(ohlcv_data[:3]):
        print(
            f"  Bar {i + 1}: {convert_timestamp_to_iso(bar.timestamp)[:10]} - Close: ${bar.close:.2f}, Volume: {bar.volume:,.0f}"
        )

    return ohlcv_data


# Run the function
apple_data = await fetch_historical_ohlcv_data()

📊 Fetched 10 OHLCV bars
📅 Date range: 2025-07-30T05:49:00+00:00 to 2025-07-30T08:08:00+00:00

🔍 First 3 bars:
  Bar 1: 2025-07-30 - Close: $1.67, Volume: 10,000
  Bar 2: 2025-07-30 - Close: $1.67, Volume: 10,000
  Bar 3: 2025-07-30 - Close: $1.67, Volume: 10,000


## Data Export to Different Formats

TVKit's `DataExporter` class provides seamless export to multiple formats including Polars DataFrames, JSON, and CSV files.

In [12]:
async def demonstrate_data_export():
    """Demonstrate different data export formats."""
    exporter = DataExporter()

    # 1. Export to Polars DataFrame
    print("📈 Exporting to Polars DataFrame ...")
    df = await exporter.to_polars(apple_data, add_analysis=False)

    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {df.columns}")
    print("\n📋 First 5 rows:")
    print(df.head())

    # 2. Export to JSON file
    print("\n💾 Exporting to JSON file...")
    json_path = await exporter.to_json(
        apple_data,
        "./tvkit_exports/apple_ohlcv_data.json",
        include_metadata=True,
        indent=2,
    )
    print(f"JSON exported to: {json_path}")

    # 3. Export to CSV file
    print("\n📊 Exporting to CSV file...")
    csv_path = await exporter.to_csv(
        apple_data,
        "./tvkit_exports/apple_ohlcv_data.csv",
        include_metadata=True,
        timestamp_format="iso",
    )
    print(f"CSV exported to: {csv_path}")

    return df


# Print Out of DataFrame
df = await demonstrate_data_export()

📈 Exporting to Polars DataFrame ...
DataFrame shape: (10, 6)
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']

📋 First 5 rows:
shape: (5, 6)
┌─────────────────────┬──────┬──────┬──────┬───────┬─────────┐
│ timestamp           ┆ open ┆ high ┆ low  ┆ close ┆ volume  │
│ ---                 ┆ ---  ┆ ---  ┆ ---  ┆ ---   ┆ ---     │
│ str                 ┆ f64  ┆ f64  ┆ f64  ┆ f64   ┆ f64     │
╞═════════════════════╪══════╪══════╪══════╪═══════╪═════════╡
│ 2025-07-30T12:49:00 ┆ 1.67 ┆ 1.67 ┆ 1.67 ┆ 1.67  ┆ 10000.0 │
│ 2025-07-30T13:15:00 ┆ 1.67 ┆ 1.67 ┆ 1.67 ┆ 1.67  ┆ 10000.0 │
│ 2025-07-30T13:30:00 ┆ 1.67 ┆ 1.67 ┆ 1.67 ┆ 1.67  ┆ 10000.0 │
│ 2025-07-30T13:34:00 ┆ 1.67 ┆ 1.67 ┆ 1.67 ┆ 1.67  ┆ 30000.0 │
│ 2025-07-30T13:38:00 ┆ 1.67 ┆ 1.67 ┆ 1.67 ┆ 1.67  ┆ 4000.0  │
└─────────────────────┴──────┴──────┴──────┴───────┴─────────┘

💾 Exporting to JSON file...
JSON exported to: tvkit_exports/apple_ohlcv_data.json

📊 Exporting to CSV file...
CSV exported to: tvkit_exports/apple_oh

## Multi-Symbol Data Comparison

Let's fetch data for multiple symbols and compare their performance.

In [13]:
async def compare_multiple_symbols():
    """Fetch and compare data for multiple symbols."""
    symbols = [
        "NASDAQ:AAPL",  # Apple
        "NASDAQ:GOOGL",  # Google
        "NASDAQ:MSFT",  # Microsoft
        "NYSE:TSLA",  # Tesla
    ]

    symbol_data = {}

    print("🔄 Fetching data for multiple symbols...")

    async with OHLCV() as ohlcv:
        for symbol in symbols:
            try:
                print(f"  📥 Fetching {symbol}...")
                data = await ohlcv.get_historical_ohlcv(
                    exchange_symbol=symbol,
                    interval="1D",
                    bars_count=30,  # Last 30 days
                )
                symbol_data[symbol] = data
                print(f"    ✅ Got {len(data)} bars")
            except Exception as e:
                print(f"    ❌ Failed to fetch {symbol}: {e}")

    # Calculate performance metrics
    print("\n📊 Performance Summary (30-day period):")
    print("-" * 60)

    for symbol, data in symbol_data.items():
        if len(data) >= 2:
            first_close = data[0].close
            last_close = data[-1].close
            change_pct = ((last_close - first_close) / first_close) * 100

            avg_volume = sum(bar.volume for bar in data) / len(data)
            max_high = max(bar.high for bar in data)
            min_low = min(bar.low for bar in data)

            print(
                f"{symbol:12} | Change: {change_pct:+6.2f}% | "
                f"Range: ${min_low:.2f}-${max_high:.2f} | "
                f"Avg Vol: {avg_volume:,.0f}"
            )

    return symbol_data


# Run multi-symbol comparison
multi_symbol_data = await compare_multiple_symbols()

🔄 Fetching data for multiple symbols...
  📥 Fetching NASDAQ:AAPL...
    ✅ Got 30 bars
  📥 Fetching NASDAQ:GOOGL...
    ✅ Got 30 bars
  📥 Fetching NASDAQ:MSFT...
    ✅ Got 30 bars
  📥 Fetching NYSE:TSLA...
    ❌ Failed to fetch NYSE:TSLA: Invalid exchange:symbol 'NYSE:TSLA' after 3 attempts

📊 Performance Summary (30-day period):
------------------------------------------------------------
NASDAQ:AAPL  | Change:  +7.70% | Range: $195.07-$216.23 | Avg Vol: 50,201,565
NASDAQ:GOOGL | Change: +11.24% | Range: $162.00-$197.95 | Avg Vol: 41,594,066
NASDAQ:MSFT  | Change:  +7.38% | Range: $472.51-$518.29 | Avg Vol: 18,052,562


## Cryptocurrency and Forex Data

TVKit supports various asset classes including cryptocurrencies and forex pairs.

In [14]:
async def fetch_crypto_and_forex_data():
    """Demonstrate fetching cryptocurrency and forex data."""

    # Different asset classes
    symbols = {
        "Cryptocurrency": [
            "BINANCE:BTCUSDT",  # Bitcoin
            "BINANCE:ETHUSDT",  # Ethereum
            "BINANCE:ADAUSDT",  # Cardano
        ],
        "Forex": [
            "FX_IDC:EURUSD",  # EUR/USD
            "FX_IDC:GBPUSD",  # GBP/USD
            "FX_IDC:USDJPY",  # USD/JPY
        ],
    }

    all_data = {}

    async with OHLCV() as ohlcv:
        for category, symbol_list in symbols.items():
            print(f"\n📊 Fetching {category} Data:")
            print("-" * 40)

            category_data = {}

            for symbol in symbol_list:
                try:
                    print(f"  📥 {symbol}...")
                    data = await ohlcv.get_historical_ohlcv(
                        exchange_symbol=symbol,
                        interval="240",  # 4-hour intervals
                        bars_count=50,
                    )
                    category_data[symbol] = data

                    # Show latest price
                    latest = data[-1]
                    print(
                        f"    ✅ Latest: ${latest.close:.6f} (Vol: {latest.volume:,.0f})"
                    )

                except Exception as e:
                    print(f"    ❌ Failed: {e}")

            all_data[category] = category_data

    # Calculate volatility for each asset
    print("\n📈 Volatility Analysis (4-hour intervals, last 50 bars):")
    print("-" * 60)

    for category, category_data in all_data.items():
        print(f"\n{category}:")
        for symbol, data in category_data.items():
            if len(data) > 1:
                # Calculate price volatility (standard deviation of returns)
                returns = []
                for i in range(1, len(data)):
                    ret = (data[i].close - data[i - 1].close) / data[i - 1].close
                    returns.append(ret)

                if returns:
                    volatility = (
                        sum((r - sum(returns) / len(returns)) ** 2 for r in returns)
                        / len(returns)
                    ) ** 0.5
                    volatility_pct = volatility * 100

                    print(
                        f"  {symbol:20} | Volatility: {volatility_pct:.3f}% | Latest: ${data[-1].close:.6f}"
                    )

    return all_data


# Fetch crypto and forex data
crypto_forex_data = await fetch_crypto_and_forex_data()


📊 Fetching Cryptocurrency Data:
----------------------------------------
  📥 BINANCE:BTCUSDT...
    ✅ Latest: $117832.570000 (Vol: 1,898)
  📥 BINANCE:ETHUSDT...
    ✅ Latest: $3777.690000 (Vol: 49,354)
  📥 BINANCE:ADAUSDT...
    ✅ Latest: $0.765100 (Vol: 29,294,649)

📊 Fetching Forex Data:
----------------------------------------
  📥 FX_IDC:EURUSD...
    ✅ Latest: $1.147160 (Vol: 35,549)
  📥 FX_IDC:GBPUSD...
    ✅ Latest: $1.328300 (Vol: 33,648)
  📥 FX_IDC:USDJPY...
    ✅ Latest: $148.969000 (Vol: 40,154)

📈 Volatility Analysis (4-hour intervals, last 50 bars):
------------------------------------------------------------

Cryptocurrency:
  BINANCE:BTCUSDT      | Volatility: 0.546% | Latest: $117832.570000
  BINANCE:ETHUSDT      | Volatility: 1.196% | Latest: $3777.690000
  BINANCE:ADAUSDT      | Volatility: 1.577% | Latest: $0.765100

Forex:
  FX_IDC:EURUSD        | Volatility: 0.193% | Latest: $1.147160
  FX_IDC:GBPUSD        | Volatility: 0.173% | Latest: $1.328300
  FX_IDC:USDJPY  

## Real-time Data Streaming (Limited Demo)

⚠️ **Note**: Real-time streaming is demonstrated with a limited time window to prevent infinite loops in the notebook.

In [7]:
async def limited_realtime_demo():
    """Demonstrate real-time streaming with a time limit."""

    print("🚀 Starting limited real-time data stream (30 seconds)...")
    print("Symbol: BINANCE:BTCUSDT (Bitcoin)")
    print("-" * 50)

    start_time = asyncio.get_event_loop().time()
    timeout_seconds = 30  # Limit to 30 seconds
    bar_count = 0

    try:
        async with OHLCV() as ohlcv:
            async for bar in ohlcv.get_ohlcv("BINANCE:BTCUSDT", interval="1"):
                # Check timeout
                if asyncio.get_event_loop().time() - start_time > timeout_seconds:
                    print(f"\n⏰ Demo timeout reached ({timeout_seconds}s)")
                    break

                bar_count += 1
                timestamp_str = convert_timestamp_to_iso(bar.timestamp)

                print(
                    f"📊 Bar {bar_count}: {timestamp_str} | "
                    f"Close: ${bar.close:,.2f} | "
                    f"Volume: {bar.volume:,.0f}"
                )

                # Also limit by number of bars
                if bar_count >= 10:
                    print(f"\n📈 Received {bar_count} bars, stopping demo")
                    break

    except Exception as e:
        print(f"❌ Streaming error: {e}")

    print(f"\n✅ Real-time demo completed. Received {bar_count} bars.")


# Run limited real-time demo
await limited_realtime_demo()

🚀 Starting limited real-time data stream (30 seconds)...
Symbol: BINANCE:BTCUSDT (Bitcoin)
--------------------------------------------------
📊 Bar 1: 2025-07-30T06:54:00+00:00 | Close: $118,288.00 | Volume: 1
📊 Bar 2: 2025-07-30T06:55:00+00:00 | Close: $118,307.81 | Volume: 5
📊 Bar 3: 2025-07-30T06:56:00+00:00 | Close: $118,318.16 | Volume: 2
📊 Bar 4: 2025-07-30T06:57:00+00:00 | Close: $118,318.16 | Volume: 2
📊 Bar 5: 2025-07-30T06:58:00+00:00 | Close: $118,318.16 | Volume: 2
📊 Bar 6: 2025-07-30T06:59:00+00:00 | Close: $118,287.31 | Volume: 7
📊 Bar 7: 2025-07-30T07:00:00+00:00 | Close: $118,287.31 | Volume: 3
📊 Bar 8: 2025-07-30T07:01:00+00:00 | Close: $118,306.00 | Volume: 7
📊 Bar 9: 2025-07-30T07:02:00+00:00 | Close: $118,308.56 | Volume: 1
📊 Bar 10: 2025-07-30T07:03:00+00:00 | Close: $118,311.72 | Volume: 1

📈 Received 10 bars, stopping demo

✅ Real-time demo completed. Received 10 bars.


## Error Handling and Best Practices

Demonstration of proper error handling and best practices when working with TVKit.

In [8]:
async def demonstrate_error_handling():
    """Show proper error handling techniques with TVKit."""

    print("🛡️  Error Handling and Best Practices")
    print("=" * 45)

    # 1. Handle invalid symbols gracefully
    print("\n1️⃣  Invalid Symbol Handling:")
    invalid_symbols = ["INVALID:SYMBOL", "BADEXCHANGE:BADSTOCK"]

    async with OHLCV() as ohlcv:
        for symbol in invalid_symbols:
            try:
                print(f"  📥 Attempting to fetch {symbol}...")
                data = await ohlcv.get_historical_ohlcv(
                    exchange_symbol=symbol, interval="1D", bars_count=10
                )
                print(f"    ✅ Success: Got {len(data)} bars")
            except Exception as e:
                print(f"    ❌ Expected error: {type(e).__name__}: {e}")

    # 2. Handle network timeouts and connection issues
    print("\n2️⃣  Connection Resilience:")

    try:
        async with OHLCV() as ohlcv:
            # This should work normally
            data = await ohlcv.get_historical_ohlcv(
                exchange_symbol="NASDAQ:AAPL", interval="1D", bars_count=5
            )
            print(f"    ✅ Successfully fetched {len(data)} bars")
    except Exception as e:
        print(f"    ❌ Connection error: {e}")

    # 3. Export error handling
    print("\n3️⃣  Export Error Handling:")

    try:
        exporter = DataExporter()

        # Try to export to an invalid path
        result = await exporter.export_ohlcv_data(
            apple_data[:5],  # Use small subset
            ExportFormat.JSON,
            file_path="/invalid/path/cannot_write_here.json",
        )

        if result.success:
            print("    ✅ Export successful")
        else:
            print(f"    ❌ Export failed: {result.error_message}")

    except Exception as e:
        print(f"    ❌ Export exception: {type(e).__name__}: {e}")

    # 4. Best practices summary
    print("\n💡 Best Practices Summary:")
    print("   • Always use async context managers (async with)")
    print("   • Handle symbol validation errors gracefully")
    print("   • Set appropriate timeouts for real-time streams")
    print("   • Check export results for success status")
    print("   • Use try-except blocks for robust error handling")
    print("   • Validate data before processing")


# Demonstrate error handling
await demonstrate_error_handling()

🛡️  Error Handling and Best Practices

1️⃣  Invalid Symbol Handling:
  📥 Attempting to fetch INVALID:SYMBOL...
    ❌ Expected error: ValueError: Invalid exchange:symbol 'INVALID:SYMBOL' after 3 attempts
  📥 Attempting to fetch BADEXCHANGE:BADSTOCK...
    ❌ Expected error: ValueError: Invalid exchange:symbol 'BADEXCHANGE:BADSTOCK' after 3 attempts

2️⃣  Connection Resilience:


2025-07-30 14:03:49,347 - ERROR - Failed to export OHLCV data to JSON: [Errno 30] Read-only file system: '/invalid'


    ✅ Successfully fetched 5 bars

3️⃣  Export Error Handling:
    ❌ Export failed: [Errno 30] Read-only file system: '/invalid'

💡 Best Practices Summary:
   • Always use async context managers (async with)
   • Handle symbol validation errors gracefully
   • Set appropriate timeouts for real-time streams
   • Check export results for success status
   • Use try-except blocks for robust error handling
   • Validate data before processing


## Summary

This notebook has demonstrated the comprehensive capabilities of TVKit:

### ✅ Completed Examples

- **Basic OHLCV Data Fetching** - Retrieved historical market data for Apple stock
- **Multi-format Data Export** - Exported to Polars DataFrame, JSON, and CSV formats
- **Multi-symbol Operations** - Compared performance across multiple stocks
- **Cryptocurrency & Forex** - Demonstrated support for various asset classes
- **Macro Liquidity Indicators** - Accessed INDEX:NDFI and USI:PCC for quantitative analysis
- **Quantitative Integration** - Showed integration with systematic trading models
- **Real-time Streaming** - Limited demo of live data streaming
- **Error Handling** - Best practices for robust applications

### 🔧 Key Features Highlighted

- **Async Architecture** - All operations use modern async/await patterns
- **Type Safety** - Comprehensive Pydantic models for data validation
- **Multiple Asset Classes** - Stocks, crypto, forex, and macro indicators
- **Flexible Export System** - Support for Polars, JSON, CSV with custom options
- **Real-time Capabilities** - WebSocket streaming for live market data
- **Quantitative Analysis** - Tools for systematic trading and risk management
- **Macro Indicators** - Access to essential liquidity and breadth metrics
- **Error Resilience** - Robust error handling and validation

### 📊 Macro Indicators Covered

- **INDEX:NDFI** - Net Demand For Income indicator for market breadth analysis
- **USI:PCC** - Put/Call Ratio for sentiment and liquidity analysis
- **Quantitative Integration** - Examples for systematic trading strategies
- **Risk Management** - Regime detection and portfolio optimization

### 📚 Next Steps

- Explore the full [TVKit documentation](https://github.com/your-repo/tvkit)
- Check out additional examples in the `examples/` directory
- Review the API reference for advanced features
- Consider integrating TVKit into your financial analysis workflows
- Implement macro indicators in your quantitative trading models

**Happy Trading! 📈**

## Macro Liquidity and Market Breadth Indicators

This section demonstrates accessing macro liquidity and market breadth indicators that are essential for quantitative liquidity models, macro regime detection, and systematic trading strategies.

### Key Indicators Covered:
- **INDEX:NDFI** - Net Demand For Income (Market Breadth Indicator)
- **USI:PCC** - Put/Call Ratio (Liquidity and Sentiment Indicator)

These indicators are widely used in professional research for:
- **Liquidity Regime Analysis** - Understanding market liquidity conditions
- **Risk Management** - Macro trend detection and regime changes  
- **Portfolio Optimization** - Systematic trading strategy development
- **Market Timing** - Entry/exit signal generation

In [ ]:
async def fetch_macro_liquidity_indicators():
    """
    Fetch macro liquidity and market breadth indicators.

    These indicators are essential for:
    - Macro liquidity regime detection
    - Market breadth analysis
    - Systematic trading strategies
    - Risk management and portfolio optimization
    """

    # Define macro indicators with descriptions
    macro_indicators = {
        "INDEX:NDFI": {
            "name": "Net Demand For Income",
            "description": "Market breadth indicator measuring income-seeking demand",
            "use_case": "Liquidity regime detection, macro trend analysis",
        },
        "USI:PCC": {
            "name": "Put/Call Ratio",
            "description": "Options sentiment and liquidity indicator",
            "use_case": "Market sentiment, volatility prediction, contrarian signals",
        },
    }

    indicator_data = {}

    print("🎯 Fetching Macro Liquidity and Market Breadth Indicators")
    print("=" * 65)

    async with OHLCV() as ohlcv:
        for symbol, info in macro_indicators.items():
            try:
                print(f"\n📊 Fetching {info['name']} ({symbol})...")
                print(f"   📝 Description: {info['description']}")
                print(f"   🎯 Use Case: {info['use_case']}")

                # Fetch historical data - using daily intervals for macro analysis
                data = await ohlcv.get_historical_ohlcv(
                    exchange_symbol=symbol,
                    interval="1D",  # Daily data for macro analysis
                    bars_count=100,  # ~3-4 months of data
                )

                indicator_data[symbol] = {"data": data, "info": info}

                # Display latest values and basic statistics
                if data:
                    latest = data[-1]
                    earliest = data[0]

                    # Calculate some basic statistics
                    values = [bar.close for bar in data]
                    avg_value = sum(values) / len(values)
                    max_value = max(values)
                    min_value = min(values)

                    # Calculate volatility (standard deviation)
                    variance = sum((x - avg_value) ** 2 for x in values) / len(values)
                    volatility = variance**0.5

                    print(f"   ✅ Successfully fetched {len(data)} bars")
                    print(
                        f"   📅 Data range: {convert_timestamp_to_iso(earliest.timestamp)[:10]} to {convert_timestamp_to_iso(latest.timestamp)[:10]}"
                    )
                    print(f"   📈 Latest value: {latest.close:.6f}")
                    print(
                        f"   📊 Statistics: Min={min_value:.6f}, Max={max_value:.6f}, Avg={avg_value:.6f}"
                    )
                    print(f"   📉 Volatility: {volatility:.6f}")

                else:
                    print("   ❌ No data received")

            except Exception as e:
                print(f"   ❌ Error fetching {symbol}: {type(e).__name__}: {e}")
                indicator_data[symbol] = {"error": str(e), "info": info}

    return indicator_data


# Fetch macro indicators
macro_data = await fetch_macro_liquidity_indicators()

### Macro Indicator Analysis and Export

Export the macro indicators to various formats for further analysis and integrate them with quantitative models.

In [ ]:
async def analyze_and_export_macro_indicators(macro_data):
    """
    Analyze macro indicators and export to multiple formats.

    This demonstrates how to:
    - Process macro liquidity indicators for quantitative analysis
    - Export data for integration with systematic trading models
    - Calculate key metrics for liquidity regime detection
    """

    print("🔬 Analyzing Macro Indicators for Quantitative Models")
    print("=" * 55)

    exporter = DataExporter()
    analysis_results = {}

    for symbol, indicator_info in macro_data.items():
        if "error" in indicator_info:
            print(f"\n❌ Skipping {symbol} due to error: {indicator_info['error']}")
            continue

        data = indicator_info["data"]
        info = indicator_info["info"]

        if not data:
            print(f"\n❌ No data available for {symbol}")
            continue

        print(f"\n📊 Analyzing {info['name']} ({symbol})")
        print("-" * 50)

        # Convert to DataFrame for analysis
        df = await exporter.to_polars(data, add_analysis=True)
        print(f"   📈 DataFrame shape: {df.shape}")

        # Calculate additional metrics for macro analysis
        if len(data) > 20:  # Ensure sufficient data
            # Recent vs Historical comparison (last 20 days vs previous 20)
            recent_values = [bar.close for bar in data[-20:]]
            historical_values = (
                [bar.close for bar in data[-40:-20]]
                if len(data) >= 40
                else [bar.close for bar in data[:-20]]
            )

            recent_avg = sum(recent_values) / len(recent_values)
            historical_avg = (
                sum(historical_values) / len(historical_values)
                if historical_values
                else recent_avg
            )

            # Trend analysis
            trend_change = (
                ((recent_avg - historical_avg) / historical_avg * 100)
                if historical_avg != 0
                else 0
            )

            # Volatility analysis
            recent_volatility = (
                sum((x - recent_avg) ** 2 for x in recent_values) / len(recent_values)
            ) ** 0.5

            # Percentile analysis (current position relative to historical range)
            all_values = [bar.close for bar in data]
            current_value = data[-1].close
            sorted_values = sorted(all_values)
            percentile = (
                sum(1 for v in sorted_values if v <= current_value) / len(sorted_values)
            ) * 100

            analysis_results[symbol] = {
                "name": info["name"],
                "current_value": current_value,
                "recent_avg": recent_avg,
                "historical_avg": historical_avg,
                "trend_change_pct": trend_change,
                "volatility": recent_volatility,
                "percentile": percentile,
                "use_case": info["use_case"],
            }

            print(f"   📈 Current Value: {current_value:.6f}")
            print(f"   📊 Recent Avg (20d): {recent_avg:.6f}")
            print(f"   📊 Historical Avg: {historical_avg:.6f}")
            print(f"   📈 Trend Change: {trend_change:+.2f}%")
            print(f"   📉 Recent Volatility: {recent_volatility:.6f}")
            print(f"   📊 Current Percentile: {percentile:.1f}%")

            # Interpretation for trading strategies
            if symbol == "INDEX:NDFI":
                if percentile > 75:
                    signal = "High income demand - Potential market strength"
                elif percentile < 25:
                    signal = "Low income demand - Potential market weakness"
                else:
                    signal = "Neutral income demand"
                print(f"   🎯 Signal: {signal}")

            elif symbol == "USI:PCC":
                if percentile > 75:
                    signal = "High put/call ratio - Potential contrarian bullish signal"
                elif percentile < 25:
                    signal = "Low put/call ratio - Potential market complacency"
                else:
                    signal = "Neutral sentiment"
                print(f"   🎯 Signal: {signal}")

        # Export individual indicator data
        try:
            # Export to CSV for systematic trading models
            csv_path = await exporter.to_csv(
                data,
                f"./tvkit_exports/macro_{symbol.replace(':', '_').lower()}_data.csv",
                include_metadata=True,
                timestamp_format="iso",
            )
            print(f"   💾 Exported to CSV: {csv_path}")

            # Export to JSON for web applications
            json_path = await exporter.to_json(
                data,
                f"./tvkit_exports/macro_{symbol.replace(':', '_').lower()}_data.json",
                include_metadata=True,
                indent=2,
            )
            print(f"   💾 Exported to JSON: {json_path}")

        except Exception as e:
            print(f"   ❌ Export error: {e}")

    # Summary analysis
    print("\n🎯 Macro Liquidity Analysis Summary")
    print("=" * 40)

    for symbol, analysis in analysis_results.items():
        print(f"\n{analysis['name']} ({symbol}):")
        print(f"  Current Level: {analysis['percentile']:.1f}th percentile")
        print(f"  Trend: {analysis['trend_change_pct']:+.2f}% (recent vs historical)")
        print(f"  Use in Models: {analysis['use_case']}")

    return analysis_results


# Analyze and export macro indicators
if macro_data:
    macro_analysis = await analyze_and_export_macro_indicators(macro_data)
else:
    print("⚠️ No macro data available for analysis")

### Integration with Quantitative Models

This section shows how to integrate macro indicators with quantitative trading strategies and risk management frameworks.

In [ ]:
def demonstrate_quantitative_integration(macro_analysis):
    """
    Demonstrate how to integrate macro indicators into quantitative models.

    This shows practical applications for:
    - Systematic trading strategies
    - Risk regime detection
    - Portfolio allocation models
    - Market timing systems
    """

    print("🧮 Quantitative Model Integration Examples")
    print("=" * 45)

    if not macro_analysis:
        print("⚠️ No macro analysis data available for integration examples")
        return

    # Example 1: Liquidity Regime Classification
    print("\n1️⃣ Liquidity Regime Classification")
    print("-" * 35)

    for symbol, analysis in macro_analysis.items():
        regime = "UNKNOWN"
        confidence = 0

        if symbol == "INDEX:NDFI":
            # NDFI-based liquidity regime detection
            percentile = analysis["percentile"]
            trend = analysis["trend_change_pct"]

            if percentile > 75 and trend > 0:
                regime = "HIGH_LIQUIDITY_EXPANDING"
                confidence = 0.85
            elif percentile > 60:
                regime = "HIGH_LIQUIDITY_STABLE"
                confidence = 0.70
            elif percentile < 25 and trend < 0:
                regime = "LOW_LIQUIDITY_CONTRACTING"
                confidence = 0.80
            elif percentile < 40:
                regime = "LOW_LIQUIDITY_STABLE"
                confidence = 0.65
            else:
                regime = "NEUTRAL_LIQUIDITY"
                confidence = 0.50

        elif symbol == "USI:PCC":
            # Put/Call ratio sentiment analysis
            percentile = analysis["percentile"]

            if percentile > 80:
                regime = "EXTREME_FEAR"
                confidence = 0.85
            elif percentile > 60:
                regime = "ELEVATED_FEAR"
                confidence = 0.70
            elif percentile < 20:
                regime = "EXTREME_COMPLACENCY"
                confidence = 0.85
            elif percentile < 40:
                regime = "LOW_FEAR"
                confidence = 0.70
            else:
                regime = "NEUTRAL_SENTIMENT"
                confidence = 0.50

        print(f"   {analysis['name']} ({symbol}):")
        print(f"   📊 Regime: {regime}")
        print(f"   🎯 Confidence: {confidence:.1%}")
        print(f"   📈 Current Level: {analysis['percentile']:.1f}th percentile")

    # Example 2: Risk Management Signals
    print("\n2️⃣ Risk Management Framework")
    print("-" * 30)

    # Combine indicators for risk assessment
    risk_score = 0
    signal_count = 0

    for symbol, analysis in macro_analysis.items():
        if symbol == "INDEX:NDFI":
            # Low NDFI = higher risk
            if analysis["percentile"] < 25:
                risk_score += 2
            elif analysis["percentile"] < 50:
                risk_score += 1
            signal_count += 1

        elif symbol == "USI:PCC":
            # Extreme levels indicate higher volatility risk
            if analysis["percentile"] > 75 or analysis["percentile"] < 25:
                risk_score += 1
            signal_count += 1

    if signal_count > 0:
        avg_risk = risk_score / signal_count

        if avg_risk >= 1.5:
            risk_level = "HIGH"
            portfolio_action = "Reduce position sizes, increase cash allocation"
        elif avg_risk >= 0.75:
            risk_level = "MEDIUM"
            portfolio_action = "Moderate position sizing, maintain diversification"
        else:
            risk_level = "LOW"
            portfolio_action = "Normal position sizing, consider growth allocation"

        print(
            f"   📊 Combined Risk Score: {risk_score}/{signal_count * 2} ({avg_risk:.2f})"
        )
        print(f"   ⚠️ Risk Level: {risk_level}")
        print(f"   🎯 Suggested Action: {portfolio_action}")

    # Example 3: Signal Generation for Trading
    print("\n3️⃣ Trading Signal Generation")
    print("-" * 30)

    signals = []

    for symbol, analysis in macro_analysis.items():
        if symbol == "INDEX:NDFI":
            if analysis["percentile"] < 25 and analysis["trend_change_pct"] > 5:
                signals.append("NDFI Reversal: Potential bullish divergence")
            elif analysis["percentile"] > 75 and analysis["trend_change_pct"] < -5:
                signals.append("NDFI Peak: Potential bearish reversal")

        elif symbol == "USI:PCC":
            if analysis["percentile"] > 80:
                signals.append("PCC Extreme Fear: Contrarian bullish opportunity")
            elif analysis["percentile"] < 20:
                signals.append("PCC Complacency: Monitor for volatility increase")

    if signals:
        for i, signal in enumerate(signals, 1):
            print(f"   {i}. {signal}")
    else:
        print("   📊 No clear trading signals detected")

    # Example 4: Model Integration Code Template
    print("\n4️⃣ Code Template for Systematic Models")
    print("-" * 40)

    template = '''
# Example integration with systematic trading model
def update_model_with_macro_indicators(macro_data):
    """
    Template for integrating macro indicators into systematic models.
    """
    
    # Extract indicator values
    ndfi_percentile = macro_data.get("INDEX:NDFI", {}).get("percentile", 50)
    pcc_percentile = macro_data.get("USI:PCC", {}).get("percentile", 50)
    
    # Regime detection logic
    liquidity_regime = classify_liquidity_regime(ndfi_percentile)
    sentiment_regime = classify_sentiment_regime(pcc_percentile)
    
    # Adjust model parameters based on regime
    if liquidity_regime == "LOW_LIQUIDITY":
        position_sizing_multiplier = 0.5  # Reduce positions
        volatility_target = 0.10  # Lower vol target
    else:
        position_sizing_multiplier = 1.0
        volatility_target = 0.15
    
    return {
        "position_sizing": position_sizing_multiplier,
        "volatility_target": volatility_target,
        "regime_signals": [liquidity_regime, sentiment_regime]
    }
    '''

    print(template)

    return {
        "risk_assessment": {
            "risk_score": risk_score if "risk_score" in locals() else 0,
            "risk_level": risk_level if "risk_level" in locals() else "UNKNOWN",
        },
        "trading_signals": signals,
        "regime_classification": {
            symbol: regime for symbol, analysis in macro_analysis.items()
        },
    }


# Run quantitative integration examples
if "macro_analysis" in locals() and macro_analysis:
    quant_results = demonstrate_quantitative_integration(macro_analysis)
else:
    print("⚠️ Running integration examples with sample data...")
    # Provide example for demonstration
    sample_analysis = {
        "INDEX:NDFI": {
            "name": "Net Demand For Income",
            "percentile": 65.0,
            "trend_change_pct": 2.5,
        },
        "USI:PCC": {
            "name": "Put/Call Ratio",
            "percentile": 75.0,
            "trend_change_pct": -1.2,
        },
    }
    quant_results = demonstrate_quantitative_integration(sample_analysis)